<a href="https://colab.research.google.com/github/tony-wade/Reverse-Engineering/blob/main/DSView%20-%20eMMC%20decoder/eMMC_Data_Parallel_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##
## This file is part of the libsigrokdecode project.
##
## Copyright (C) 2011 Gareth McMullin <gareth@blacksphere.co.nz>
## Copyright (C) 2012-2014 Uwe Hermann <uwe@hermann-uwe.de>
## Copyright (C) 2022 DreamSourceLab <support@dreamsourcelab.com>
## Copyright (C) 2024 Wade Wu <wdwu4work@gmail.com>
##
## This program is free software; you can redistribute it and/or modify
## it under the terms of the GNU General Public License as published by
## the Free Software Foundation; either version 2 of the License, or
## (at your option) any later version.
##
## This program is distributed in the hope that it will be useful,
## but WITHOUT ANY WARRANTY; without even the implied warranty of
## MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
## GNU General Public License for more details.
##
## You should have received a copy of the GNU General Public License
## along with this program; if not, see <http://www.gnu.org/licenses/>.
##

## text block fill color table:
##  [#EF2929,#F66A32,#FCAE3E,#FBCA47,#FCE94F,#CDF040,#8AE234,#4EDC44,#55D795,#64D1D2
##  ,#729FCF,#D476C4,#9D79B9,#AD7FA8,#C2629B,#D7476F]

import sigrokdecode as srd
from common.srdhelper import bitpack  # binary to decimal



def channel_list(num_channels):
    l = [{'id': 'clk', 'name': 'CLK/DS', 'desc': 'Clock line'}]
    for i in range(num_channels):
        d = {'id': 'd%d' % i, 'name': 'D%d' % i, 'desc': 'Data line %d' % i}
        l.append(d)
    return tuple(l)

NUM_CHANNELS = 8

class ChannelError(Exception):
    pass

class Decoder(srd.Decoder):
    api_version = 3
    id = 'eMMC_parallel'
    name = 'eMMC Parallel'
    longname = 'eMMC Parallel data decoder'
    desc = 'eMMC parallel data decoder.'
    license = 'gplv2+'
    inputs = ['logic']
    outputs = ['parallel']
    tags = ['Util']
    optional_channels = channel_list(NUM_CHANNELS)
    options = (
        {'id': 'clock_edge', 'desc': 'Clock edge for sampling',
            'default': 'rising', 'values': ('rising', 'falling', 'dual'), 'idn':'dec_parallel_opt_clock_edge'},
        {'id': 'wordsize', 'desc': 'Data wordsize (ex. 4 bus choose 2)',
            'default': 0, 'idn':'dec_parallel_opt_wordsize'},
        {'id': 'config', 'desc':'eMMC / plain parallel data',
            'default': 'eMMC', 'values':('eMMC','Normal'), 'idn':'dec_parallel_opt_config'},
        {'id': 'endianness', 'desc': 'Data endianness',
            'default': 'big', 'values': ('little', 'big'), 'idn':'dec_parallel_opt_endianness'},
        {'id': 'mode', 'desc':'MCU opcode detect mode',
            'default': 'Disable', 'values':('Disable','8051'), 'idn':'dec_parallel_opt_opcode'},
    )
    annotations = (
        ('items', 'Items'),
        ('words', 'Words'),
    )
    annotation_rows = (
        ('items', 'Items', (0,)),
        ('words', 'Words', (1,)),
    )

    def __init__(self):
        self.reset()

    def reset(self):
        self.items = []
        self.saved_item = None
        self.saved_word = None
        self.ss_word = self.es_word = None
        self.first_bit = True
        self.first_word = True
        self.have_clock = True
        self.prv_dex = 0
        self.num_item_bits = None
        self.state = 'start'
        self.opcode_dict = {
            # key needs to be the same in 'mode'
            '8051': set([128, 1, 33, 65, 97, 113, 161, 193, 225, 2]),  # SJMP, AJMP, LJMP
            }



    def start(self):
        self.out_python = self.register(srd.OUTPUT_PYTHON)
        self.out_ann = self.register(srd.OUTPUT_ANN)

    def putpw(self, data):
        self.put(self.ss_word, self.es_word, self.out_python, data)

    def putw(self, data):
        self.put(self.ss_word, self.es_word, self.out_ann, data)

    def put_ann(self, s, e, data):
        self.put(s, e, self.out_ann, data)

    def put_py(self, s, e, data):
        self.put(s, e, self.out_python, data)

    def handle_bits(self, item):
        # If a word was previously accumulated, then emit its annotation
        # now after its end samplenumber became available.
        cur_dex = self.samplenum

        # Defer annotations for individual items until the next sample
        # is taken, and the previous sample's end samplenumber has
        # become available.
        if self.first_bit:
            # Save the start sample and item for later (no output yet).
            if not self.have_clock:
                self.put_py(self.prv_dex, cur_dex, ['ITEM', self.saved_item])
                self.put_ann(self.prv_dex, cur_dex, [0, [self.fmt_item.format(self.saved_item)]])

            self.first_bit = False
            self.saved_item = item
        else:
            # Output the saved item (from the last CLK edge to the current).
            self.put_py(self.prv_dex, cur_dex, ['ITEM', self.saved_item])
            self.put_ann(self.prv_dex, cur_dex, [0, [self.fmt_item.format(self.saved_item)]])
            self.saved_item = item


        self.prv_dex = cur_dex
        self.handle_word(item, cur_dex)  # Send to 'WORD' line


    def handle_word(self, item, cur_dex):
        if self.saved_word is not None:
            if self.options['wordsize'] > 0:
                self.es_word = cur_dex
                self.putw([1, [self.fmt_word.format(self.saved_word)]])
                self.putpw(['WORD', self.saved_word])
            self.saved_word = None

        if item is None:
            return

         # Get as many items as the configured wordsize specifies.
        if not self.items:
            self.ss_word = cur_dex

        self.items.append(item)
        ws = self.options['wordsize']

        if len(self.items) < ws:
            return

        # Collect words and prepare annotation details, but defer emission
        # until the end samplenumber becomes available.
        endian = self.options['endianness']

        if endian == 'big':
            self.items.reverse()

        word = sum([self.items[i] << (i * self.num_item_bits) for i in range(ws)])

        if self.options['mode'] == 'Disable' or not self.first_word:
            self.saved_word = word
            self.items = []
        elif word in self.opcode_dict.get(self.options['mode'],[]):
            self.first_word = False
            self.saved_word = word
            self.items = []
        else:
            self.state = 'idle'     # Wait NAK to initalize
            self.first_word = False
            self.items = []



    def end(self):
        cur_dex = self.last_samplenum
        #the last annotation
        if self.saved_item != None:
            self.put_py(self.prv_dex, cur_dex, ['ITEM', self.saved_item])
            self.put_ann(self.prv_dex, cur_dex, [0, [self.fmt_item.format(self.saved_item)]])
            self.handle_word(None, cur_dex)


    def decode(self):
        idx_channels = [
            idx if self.has_channel(idx) else None
            for idx in range(len(self.optional_channels))
        ]

        has_channels = [idx for idx in idx_channels if idx is not None]
        if not has_channels:
            raise ChannelError('At least one channel has to be supplied.')
        max_connected = max(has_channels)

        # fundamental setting
        self.have_clock = self.has_channel(0)
        have_clock = self.have_clock
        if max_connected == 3:
            block_size *= 2     # 4 bus need 2 datas to transmit 1 byte

        # output setting
        self.prv_dex = self.samplenum
        idx_strip = max_connected + 1
        num_item_bits = max_connected
        num_word_items = self.options['wordsize']
        num_word_bits = num_item_bits * num_word_items
        num_digits = (num_item_bits + 3) // 4
        self.fmt_item = "@{{:0{}X}}".format(num_digits)
        num_digits = (num_word_bits + 3) // 4
        self.fmt_word = "@{{:0{}X}}".format(num_digits)
        self.num_item_bits = num_item_bits


        # clk setting
        if have_clock:
            edge = self.options['clock_edge']
            if edge == 'rising':
                conds = {0: 'r'}
            elif edge == 'falling':
                conds = {0: 'f'}
            elif edge == 'dual':
                # Handle both rising and falling edges
                conds = {0: 'e'}
        else:
            conds = [{idx: 'e'} for idx in has_channels]

        data_count = 0
        data_high_len = 0
        NAC_clk = 6*2 if edge=='dual' else 3*2    # blank cycle between blocks
        ddr_start_bit = False
        self.state = 'start'
        stop_flag = False

        while True:
            (clk, d0, d1, d2, d3, d4, d5, d6, d7) = self.wait(conds)
            pins = (clk, d0, d1, d2, d3, d4, d5, d6, d7)
            bits = [0 if idx is None else pins[idx] for idx in idx_channels]
            valid_bits = bits[1:idx_strip]
            item = bitpack(valid_bits)

            if self.options['config'] == 'Normal':
                # Normal parallel data decoding
                self.handle_bits(item)
                continue

            # Correctly divide each block
            if data_high_len >= NAC_clk and num_word_items > 0:
                if data_count % num_word_items != 0:
                    # fill data until divisible
                    self.handle_bits(item)
                    data_count += 1
                    continue
                else:
                    # Reset the process for the next block
                    data_count = 0
                    data_high_len = 0
                    ddr_start_bit = False
                    self.state = 'start'
                    self.first_word = True
                    continue


            if self.state == 'start':
                # Step 1: Start condition
                if ddr_start_bit:
                    self.state = 'data_transfer'
                elif all(bit == 0 for bit in valid_bits) and clk == 1:
                    if edge != 'dual':
                        self.state = 'data_transfer'
                    else:
                        ddr_start_bit = True
                continue

            elif self.state == 'data_transfer':
                # Step 2: Data processing
                self.handle_bits(item)
                data_count += 1


            # Stop condition
            is_data_high = all(bit == 1 for bit in valid_bits)

            if is_data_high:
                if stop_flag:
                    data_high_len += 1
                elif clk == 1:
                    stop_flag = True
                    data_high_len += 1
            else:
                data_high_len = 0
                stop_flag = False
